In [1]:
# =========================================================
# Kernel Search (KS) for sparse quadratic SVM with CoP(K)
#   - Step (13): D-Shor-Co (MOSEK Fusion)
#   - Step (17)+(18): CoP(K) w/ indicator constraints (Gurobi)
#   * Robust changes:
#       A) ensure |K_it| >= B each iteration
#       B) provide explicit feasible MIP Start to get a finite UB fast
#       C) (optional) Gurobi params to favor feasibility
# =========================================================
from __future__ import annotations
import math
import itertools
import numpy as np
from typing import List, Dict, Tuple, Optional, Iterable

# ---------- helpers ----------
def _fmt_num(x):
    return "inf" if (x is None or not np.isfinite(x)) else f"{x:.6g}"

def _fusion_set_time_limit(M, seconds: float | None, is_mip: bool = False):
    """MOSEK Fusion time limit with fallback param names across versions."""
    if seconds is None:
        return
    try:
        import mosek
        M.setSolverParam(mosek.dparam.optimizer_max_time, float(seconds))
        if is_mip:
            M.setSolverParam(mosek.dparam.mio_max_time, float(seconds))
    except Exception:
        # textual fallbacks
        M.setSolverParam("optimizerMaxTime", float(seconds))
        if is_mip:
            M.setSolverParam("mioMaxTime", float(seconds))

def _fusion_obj_value(M):
    """Get MOSEK objective value in a version-compatible way."""
    try:
        return float(M.primalObjValue())
    except Exception:
        return float(M.objectiveValue())

def _ensure_kernel_size(order: np.ndarray | List[int], K_it: List[int], B: int, extra: int = 0) -> List[int]:
    """
    Ensure |K_it| >= B (and optionally >= B+extra) by augmenting with the best-ranked
    features from 'order' that are not yet in K_it.
    """
    want = max(B + extra, len(K_it))
    Kset = set(int(j) for j in K_it)
    for idx in list(order):
        if len(Kset) >= want:
            break
        if int(idx) not in Kset:
            Kset.add(int(idx))
    return sorted(Kset)

# ---------- 0) 二次映射（显式） ----------
def quad_map(X: np.ndarray):
    """
    返回：Phi_diag (m×n), Phi_cross_list (list of length P, 每个 m-vector), pairs (索引对列表)
    二次映射采用：phi(x) = [ x_1^2, ..., x_n^2, sqrt(2)*x_1x_2, ... ]
    """
    m, n = X.shape
    Phi_diag = X ** 2
    pairs = list(itertools.combinations(range(n), 2))
    sqrt2 = np.sqrt(2.0)
    Phi_cross_list = [sqrt2 * (X[:, i] * X[:, j]) for (i, j) in pairs]
    return Phi_diag, Phi_cross_list, pairs

# ---------- 1) (13) D-Shor-Co by MOSEK (逐样本约束，避免维度问题) ----------
def solve_dshor_co_mosek(
    X: np.ndarray, y: np.ndarray, B: int, C: float,
    time_limit: Optional[float] = None, verbose: bool = False
) -> Dict:
    """
    近似实现 (13)：min 0.5*sum(Wjj) + C*sum xi
    s.t. y_i (b + sum_j w_j * x_{ij}^2) >= 1 - xi_i
         xi >= 0
         0 <= t <= 1,  sum t = n - B
         w_j^2 <= Wjj   (用 rotated QC)
    备注：margin 只用对角基（x_j^2）。
    """
    try:
        import mosek.fusion as mf
    except Exception as e:
        raise RuntimeError("需要安装 MOSEK（mosek, mosek.fusion）。") from e

    m, n = X.shape
    Phi_diag, _, _ = quad_map(X)

    M = mf.Model("DShorCo")
    if not verbose:
        M.setLogHandler(None)
    _fusion_set_time_limit(M, time_limit, is_mip=False)

    # 变量
    w  = M.variable("w",  n, mf.Domain.unbounded())
    Wd = M.variable("Wd", n, mf.Domain.greaterThan(0.0))        # diag(W)
    b  = M.variable("b",  1, mf.Domain.unbounded())
    xi = M.variable("xi", m, mf.Domain.greaterThan(0.0))
    t  = M.variable("t",  n, mf.Domain.inRange(0.0, 1.0))

    # 预算 e^T t = n - B
    M.constraint(mf.Expr.sum(t), mf.Domain.equalsTo(float(n - B)))

    # w_j^2 <= Wjj  —— 旋转二阶锥：2*x*y >= z^2，取 x=Wd[j], y=0.5, z=w[j]
    half = mf.Expr.constTerm(1, 0.5)
    for j in range(n):
        M.constraint(mf.Expr.hstack(Wd.index(j), half, w.index(j)), mf.Domain.inRotatedQCone())

    # margin：逐样本 y_i*(b + <phi_i, w>) >= 1 - xi_i
    A = mf.Matrix.dense(Phi_diag)          # m×n
    sAw = mf.Expr.mul(A, w)                # 长度 m 的表达式向量
    for i in range(m):
        lhs = mf.Expr.mul(float(y[i]), mf.Expr.add(sAw.index(i), b))  # 标量
        M.constraint(mf.Expr.sub(lhs, mf.Expr.sub(1.0, xi.index(i))), mf.Domain.greaterThan(0.0))

    # 目标：0.5*sum(Wd) + C*sum(xi)
    obj = mf.Expr.add( mf.Expr.mul(0.5, mf.Expr.sum(Wd)), mf.Expr.mul(C, mf.Expr.sum(xi)) )
    M.objective(mf.ObjectiveSense.Minimize, obj)

    M.solve()
    objval = _fusion_obj_value(M)

    sol = {
        "status": str(M.getPrimalSolutionStatus()),
        "objval": objval,
        "t": np.array(t.level()).reshape(-1),
        "w": np.array(w.level()).reshape(-1),
        "Wd": np.array(Wd.level()).reshape(-1),
    }
    M.dispose()
    return sol

# ---------- 2) CoP(K)（Gurobi：指示约束，无 Big-M，含可行 MIP Start） ----------
def build_and_solve_cop_kernel(
    X: np.ndarray, y: np.ndarray, B: int, C: float,
    feature_names: Optional[List[str]],
    kernel_set: Iterable[int],        # 当前允许“可能被选”的原始特征索引集合 K_it
    UB: Optional[float],              # 若给定，则加入 (17)：0.5||w||^2 + CΣxi <= UB
    time_limit: float = 120.0,
    mipgap: Optional[float] = None,
    verbose: bool = False,
    start_idx: Optional[List[int]] = None,   # ★ 新增：MIP Start（优先用历史最优，否则 K_it 前B）
    favor_feasible: bool = True              # ★ 可选：更有利于先找到可行解
) -> Dict:
    """
    原始 CoP(K)：使用“指示约束”实现 t_j=1=>w_j=0, (t_i=1 or t_j=1)=>w_ij=0
    仅允许 kernel_set 中的原始特征被启用（其余 t_j 固定为 1）。
    附加 (17)(18)：
      (17) 0.5||w||^2 + CΣxi <= UB   （若 UB 非空）
      (18) Σ_{j∈K_it} t_j <= |K_it| - 1  （至少选 K_it 中的一个）
    """
    import gurobipy as gp
    from gurobipy import GRB

    m, n = X.shape
    Phi_diag, Phi_cross_list, pairs = quad_map(X)
    P = len(pairs)

    model = gp.Model("CoP_KS")
    model.Params.OutputFlag = 1 if verbose else 0
    model.Params.TimeLimit = time_limit
    if mipgap is not None:
        model.Params.MIPGap = mipgap
    # 可行解友好（可选）
    if favor_feasible:
        model.Params.MIPFocus = 1      # 优先找可行解
        model.Params.Heuristics = 0.5
        model.Params.Presolve = 2
        model.Params.Cuts = 1
    model.Params.Seed = 0

    # 变量
    w_diag  = model.addVars(n, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="w_sq")
    w_cross = model.addVars(P, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="w_cross")
    b       = model.addVar(lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="b")
    xi      = model.addVars(m, lb=0.0, vtype=GRB.CONTINUOUS, name="xi")
    t       = model.addVars(n, vtype=GRB.BINARY, name="t")  # 1=禁用

    # 指示：t_j=1 ⇒ w_diag_j=0
    for j in range(n):
        t[j].BranchPriority = 10
        model.addGenConstrIndicator(t[j], 1, w_diag[j], GRB.EQUAL, 0.0, name=f"diag_zero_{j}")

    # 指示：t_i=1 or t_j=1 ⇒ w_cross_ij=0
    for k, (i, j) in enumerate(pairs):
        model.addGenConstrIndicator(t[i], 1, w_cross[k], GRB.EQUAL, 0.0, name=f"cross_i_zero_{k}")
        model.addGenConstrIndicator(t[j], 1, w_cross[k], GRB.EQUAL, 0.0, name=f"cross_j_zero_{k}")

    # 预算：sum t = n - B
    model.addConstr(gp.quicksum(t[j] for j in range(n)) == n - B, name="budget")

    # margin 逐样本
    for i in range(m):
        expr = b
        for j in range(n):
            expr += w_diag[j] * float(Phi_diag[i, j])
        for k in range(P):
            expr += w_cross[k] * float(Phi_cross_list[k][i])
        model.addConstr(float(y[i]) * expr >= 1.0 - xi[i], name=f"margin_{i}")

    # (18) 至少选 K_it 中的一个：sum t_j <= |K|-1
    K_it = sorted(set(int(j) for j in kernel_set))
    if len(K_it) > 0:
        model.addConstr(gp.quicksum(t[j] for j in K_it) <= len(K_it) - 1, name="atleast_one_in_kernel")

    # 强制：非 kernel_set 的特征一律禁用（t_j=1）
    outside = sorted(set(range(n)) - set(K_it))
    for j in outside:
        model.addConstr(t[j] == 1, name=f"fix_outside_{j}")

    # 目标与 (17) 上界约束
    quad = gp.QuadExpr()
    for j in range(n): quad += w_diag[j] * w_diag[j]
    for k in range(P): quad += w_cross[k] * w_cross[k]
    obj = 0.5 * quad + C * gp.quicksum(xi[i] for i in range(m))
    model.setObjective(obj, GRB.MINIMIZE)
    if (UB is not None) and np.isfinite(UB):
        model.addQConstr(obj <= UB + 1e-9, name="obj_ub")  # (17)

    # ---------- ★ MIP Start：强制给出可行解，确保立刻有有限 UB ----------
    # 先全部禁用，再在 K_it 中挑 B 个置 0
    for j in range(n):
        t[j].Start = 1
    seed = (start_idx if (start_idx is not None and len(start_idx)>0) else K_it)[:min(B, len(K_it))]
    for j in seed:
        t[j].Start = 0
    # 连续变量的平凡可行值：w=0, b=0, xi=1  => y_i*(0) >= 1-1 == 0  ✔
    b.Start = 0.0
    for j in range(n):
        w_diag[j].Start = 0.0
    for k in range(P):
        w_cross[k].Start = 0.0
    for i in range(m):
        xi[i].Start = 1.0
    # ---------------------------------------------------------------

    # 求解
    model.optimize()
    sol = {
        "status": model.Status,
        "runtime": model.Runtime,
        "objval": model.ObjVal if model.SolCount > 0 else None,
        "gap": model.MIPGap if model.SolCount > 0 else None,
        "selected_idx": [],
        "selected_features": [],
        "t": None
    }
    if model.SolCount > 0:
        idx = [j for j in range(n) if t[j].X < 0.5]  # t=0 ⇒ 选中
        sol["selected_idx"] = idx
        if feature_names:
            sol["selected_features"] = [feature_names[j] for j in idx]
        sol["t"] = np.array([t[j].X for j in range(n)])
    return sol

# ---------- 3) Kernel Search 主流程（含 |K_it|≥B 与 MIP Start） ----------
def kernel_search(
    X: np.ndarray, y: np.ndarray, B: int, C: float, rho: int,
    feature_names: Optional[List[str]] = None,
    # 解器参数
    mosek_time: Optional[float] = None,
    mip_time: float = 120.0,
    mip_gap: Optional[float] = None,
    verbose: bool = True
) -> Dict:
    """
    输入：X (m×n), y∈{-1,+1}^m, 预算 B, 正则 C, 参数 rho（每组大小）
    步骤：
      1) 解 D-Shor-Co（13），得到 t* 并按升序对特征排序
      2) 把排序结果均匀分成 N=ceil(n/rho) 组 K1..KN
      3) 迭代 k=1..N：
           K_it := ensure_size(K_k ∪ K_bar)  # ★ 确保 |K_it|≥B
           以历史最优作为 start_idx（否则用 K_it 前B），解 CoP(K_it) + (17)(18)
           若可行：更新 UB、K^+、K^-、K_bar
    输出：最优可行解（特征集合）与 UB
    """
    m, n = X.shape
    # 1) D-Shor-Co
    rel = solve_dshor_co_mosek(X, y, B=B, C=C, time_limit=mosek_time, verbose=verbose)
    t_relax = rel["t"]
    order = np.argsort(t_relax)  # 越小越“该选”
    if verbose:
        print(f"[D-Shor-Co] obj={_fmt_num(rel['objval'])}  t_min..max=({_fmt_num(float(t_relax.min()))}, {_fmt_num(float(t_relax.max()))})")

    # 2) 分组
    rho = max(1, min(rho, n))
    N = math.ceil(n / rho)
    groups = [order[i*rho : min((i+1)*rho, n)].tolist() for i in range(N)]
    if verbose:
        print(f"[KS] n={n}, rho={rho}  => N={N} groups (each ≤{rho})")

    # 3) 迭代
    K_bar: List[int] = []           # 累积核
    stale: Dict[int, int] = {}      # 不被选中的迭代次数
    UB = np.inf
    best = {"objval": np.inf, "selected_idx": [], "selected_features": []}

    for k in range(N):
        K_k = groups[k]
        K_it_raw = sorted(set(K_k) | set(K_bar))
        # ★ A) 确保 |K_it| ≥ B（也可给点冗余 extra）
        K_it = _ensure_kernel_size(order, K_it_raw, B, extra=max(1, B//2))

        if verbose:
            print(f"[KS] iter {k+1}/{N}  |K_k|={len(K_k)}  |K_bar|={len(K_bar)}  |K_it|={len(K_it)}  UB={_fmt_num(UB)}")

        # ★ B) 选择 MIP Start（优先历史最优，否则 K_it 前B）
        seed = best["selected_idx"] if best["selected_idx"] else K_it[:B]

        sol = build_and_solve_cop_kernel(
            X, y, B, C, feature_names=feature_names,
            kernel_set=K_it,
            UB=UB,
            time_limit=mip_time,
            mipgap=mip_gap,
            verbose=verbose,
            start_idx=seed,              # ★ 传入 MIP Start
            favor_feasible=True          # ★ 更友好可行解
        )

        if sol["objval"] is None:
            if verbose:
                print("   -> infeasible or no incumbent under limits.")
            # 若不可行，可适度扩大 K_it 或放宽 gap/time，下次迭代继续
            continue

        # 可行：更新上界与最优
        UB = min(UB, sol["objval"])
        if sol["objval"] < best["objval"]:
            best = {"objval": sol["objval"],
                    "selected_idx": sol["selected_idx"],
                    "selected_features": sol["selected_features"]}
        sel_set = set(sol["selected_idx"])

        # K^+ ：本轮 K_k 中被选的
        K_plus = [j for j in K_k if j in sel_set]
        # K^- ：K_bar 中连续两轮未被选的（这里用 stale>=1 的简化版）
        K_minus = [j for j in K_bar if j not in sel_set and stale.get(j, 0) >= 1]

        # 更新 stale 计数
        for j in K_bar:
            stale[j] = 0 if j in sel_set else stale.get(j, 0) + 1

        # 更新 K_bar
        K_bar = sorted(set(K_bar).union(K_plus) - set(K_minus))

        if verbose:
            print(f"   -> obj={sol['objval']:.6g},  |K^+|={len(K_plus)},  |K^-|={len(K_minus)},  |K_bar|={len(K_bar)}")

    if verbose:
        print("[KS] DONE.  best obj=", _fmt_num(best["objval"]), "  |S|=", len(best["selected_idx"]))
    return {
        "UB": float(UB),
        "best_obj": (float(best["objval"]) if np.isfinite(best["objval"]) else None),
        "selected_idx": list(best["selected_idx"]),
        "selected_features": list(best.get("selected_features", [])),
        "best": best  # 兼容旧用法（里面有 objval/selected_idx/selected_features）
    }

In [2]:
# --- 放在文件前面工具区 ---
def _fusion_obj_value(M):
    """取 MOSEK Fusion 的目标值，兼容不同版本接口。"""
    try:
        return float(M.primalObjValue())   # 常用
    except Exception:
        return float(M.objectiveValue())   # 备用


In [3]:
def _fmt_num(x):
    return "inf" if (x is None or not np.isfinite(x)) else f"{x:.6g}"

In [4]:
# =========================================================
# Heuristic procedure (Algorithm 3) using Kernel Search (Alg. 2)
#   (13) D-Shor-Co  .......... MOSEK Fusion (conic)
#   (19) bound M_j^l / M_j^u . MOSEK Fusion (conic) with UB
#   (14) D-Shor-Co-BigM ...... MOSEK Fusion (MISOCP, diag part for warm start)
#   CoP(K) ................... Gurobi (indicator constraints, full quadratic map)
# =========================================================
from __future__ import annotations
import math
import itertools
import numpy as np
from typing import List, Dict, Optional, Iterable

# -------------------- explicit quadratic map --------------------
def quad_map(X: np.ndarray):
    """
    phi(x) = [ x_1^2, ..., x_n^2, sqrt(2)*x_1x_2, ..., sqrt(2)*x_{n-1}x_n ]
    返回：
      Phi_diag: m×n  （平方项）
      Phi_cross_list: 长度 P 的 list，每个是 m-vector（交叉项）
      pairs: P 个 (i,j) 索引对，与 Phi_cross_list 对应
    """
    m, n = X.shape
    Phi_diag = X ** 2
    pairs = list(itertools.combinations(range(n), 2))
    sqrt2 = np.sqrt(2.0)
    Phi_cross_list = [sqrt2 * (X[:, i] * X[:, j]) for (i, j) in pairs]
    return Phi_diag, Phi_cross_list, pairs

# -------------------- objective helper --------------------
def obj_value(w_diag: np.ndarray, w_cross: np.ndarray, xi: np.ndarray, C: float) -> float:
    return 0.5 * (float(np.dot(w_diag, w_diag)) + float(np.dot(w_cross, w_cross))) + C * float(np.sum(xi))

# -------------------- Fusion time-limit helper --------------------
def _fusion_set_time_limit(M, seconds: float | None, is_mip: bool = False):
    """
    给 MOSEK Fusion 模型设定时间上限；兼容不同版本。
    - is_mip=True 时，同时对 MIP 专用和全局优化器时间都限时。
    """
    if seconds is None:
        return
    try:
        import mosek
        M.setSolverParam(mosek.dparam.optimizer_max_time, float(seconds))
        if is_mip:
            M.setSolverParam(mosek.dparam.mio_max_time, float(seconds))
    except Exception:
        M.setSolverParam("optimizerMaxTime", float(seconds))
        if is_mip:
            M.setSolverParam("mioMaxTime", float(seconds))

# =========================================================
# (13) D-Shor-Co (MOSEK) —— 使用对角二次基，足够用于 KS 排序
# =========================================================
def solve_dshor_co_mosek(X: np.ndarray, y: np.ndarray, B: int, C: float,
                         time_limit: Optional[float] = None, verbose: bool = False) -> Dict:
    import mosek.fusion as mf
    m, n = X.shape
    Phi_diag, _, _ = quad_map(X)

    M = mf.Model("DShorCo")
    if not verbose: M.setLogHandler(None)
    _fusion_set_time_limit(M, time_limit, is_mip=False)

    w  = M.variable("w",  n, mf.Domain.unbounded())
    Wd = M.variable("Wd", n, mf.Domain.greaterThan(0.0))
    b  = M.variable("b",  1, mf.Domain.unbounded())
    xi = M.variable("xi", m, mf.Domain.greaterThan(0.0))
    t  = M.variable("t",  n, mf.Domain.inRange(0.0, 1.0))

    M.constraint(mf.Expr.sum(t), mf.Domain.equalsTo(float(n - B)))

    half = mf.Expr.constTerm(1, 0.5)
    for j in range(n):
        M.constraint(mf.Expr.hstack(Wd.index(j), half, w.index(j)), mf.Domain.inRotatedQCone())

    A = mf.Matrix.dense(Phi_diag)
    sAw = mf.Expr.mul(A, w)
    for i in range(m):
        lhs = mf.Expr.mul(y[i], mf.Expr.add(sAw.index(i), b))
        M.constraint(mf.Expr.sub(lhs, mf.Expr.sub(1.0, xi.index(i))), mf.Domain.greaterThan(0.0))

    obj = mf.Expr.add(mf.Expr.mul(0.5, mf.Expr.sum(Wd)), mf.Expr.mul(C, mf.Expr.sum(xi)))
    M.objective(mf.ObjectiveSense.Minimize, obj)
    M.solve()

    objval = _fusion_obj_value(M)  # ★ 改这里

    sol = {
        "status": str(M.getPrimalSolutionStatus()),
        "objval": objval,                                 # ★ 改这里
        "t": np.array(t.level()).reshape(-1),
        "w": np.array(w.level()).reshape(-1),
        "Wd": np.array(Wd.level()).reshape(-1),
    }
    M.dispose()
    return sol

def compute_M_by_prop41(X: np.ndarray, y: np.ndarray, B: int, C: float, UB: float,
                        time_per_bound: Optional[float] = None, verbose: bool = False) -> float:
    """
    命题 4.1：M = max_j max{|M_j^l|, |M_j^u|}
    其中 M_j^l = min w_j, M_j^u = max w_j，均在 (19) 约束下求解。
    """
    n = X.shape[1]
    Ms = []
    for j in range(n):
        lo = bound_wj_mosek(X, y, B, C, UB, j, "min", time_limit=time_per_bound, verbose=verbose)
        hi = bound_wj_mosek(X, y, B, C, UB, j, "max", time_limit=time_per_bound, verbose=verbose)
        Ms.append(max(abs(lo), abs(hi)))
        if verbose:
            print(f"[Prop4.1] j={j}  lo={lo:.6g}  hi={hi:.6g}  max={Ms[-1]:.6g}")
    M = float(np.max(Ms))
    if verbose:
        print(f"[Prop4.1] M = {M:.6g}")
    return M

# =========================================================
# (1) l2-NLSVM（对角+交叉二次显式映射） —— MOSEK（凸二次）
# =========================================================
def solve_l2_nlsvm_mosek(X: np.ndarray, y: np.ndarray, C: float,
                         time_limit: Optional[float] = None, verbose: bool = False) -> Dict:
    try:
        import mosek
        import mosek.fusion as mf
    except Exception as e:
        raise RuntimeError("请先安装 MOSEK 与 mosek.fusion。") from e

    m, n = X.shape
    Phi_diag, Phi_cross_list, pairs = quad_map(X)
    P = len(pairs)
    Phi_cross = np.vstack([pc for pc in Phi_cross_list]).T if P > 0 else np.zeros((m, 0))

    M = mf.Model("l2-NLSVM")
    if not verbose:
        M.setLogHandler(None)
    _fusion_set_time_limit(M, time_limit, is_mip=False)

    w_d = M.variable("w_d", n, mf.Domain.unbounded())
    w_c = M.variable("w_c", P, mf.Domain.unbounded()) if P > 0 else None
    b   = M.variable("b", 1, mf.Domain.unbounded())
    xi  = M.variable("xi", m, mf.Domain.greaterThan(0.0))

    # margin: for each i, y_i*(b + <phi_diag_i, w_d> + <phi_cross_i, w_c>) >= 1 - xi_i
    A_d = mf.Matrix.dense(Phi_diag)
    s_d = mf.Expr.mul(A_d, w_d)  # length m
    if P > 0:
        A_c = mf.Matrix.dense(Phi_cross)
        s_c = mf.Expr.mul(A_c, w_c)  # length m
        s = mf.Expr.add(s_d, s_c)
    else:
        s = s_d
    for i in range(m):
        lhs = mf.Expr.mul(y[i], mf.Expr.add(s.index(i), b))
        M.constraint(mf.Expr.sub(lhs, mf.Expr.sub(1.0, xi.index(i))), mf.Domain.greaterThan(0.0))

    # 目标：0.5*(||w_d||^2 + ||w_c||^2) + C*sum xi
    quad_norm = mf.Expr.dot(w_d, w_d)
    if P > 0:
        quad_norm = mf.Expr.add(quad_norm, mf.Expr.dot(w_c, w_c))
    obj = mf.Expr.add(mf.Expr.mul(0.5, quad_norm), mf.Expr.mul(C, mf.Expr.sum(xi)))
    M.objective(mf.ObjectiveSense.Minimize, obj)
    M.solve()

    w_diag = np.array(w_d.level()).reshape(-1)
    w_cross = np.array(w_c.level()).reshape(-1) if P > 0 else np.zeros(P)
    bval = float(b.level()[0])
    xi_v = np.array(xi.level()).reshape(-1)
    M.dispose()
    return {
        "w_diag": w_diag, "w_cross": w_cross, "b": bval, "xi": xi_v,
        "obj": obj_value(w_diag, w_cross, xi_v, C),
        "l1_over_B": float(np.sum(np.abs(np.r_[w_diag, w_cross]))) / max(1, X.shape[1])
    }

# =========================================================
# (14) D-Shor-Co-BigM（MOSEK, MISOCP, diag-only，用作启发式起点）
# =========================================================
def solve_dshor_co_bigm_mosek(X: np.ndarray, y: np.ndarray, B: int, C: float, Mbig: float,
                              time_limit: Optional[float] = None, verbose: bool = False) -> Dict:
    import mosek.fusion as mf
    m, n = X.shape
    Phi_diag, _, _ = quad_map(X)

    M = mf.Model("DShorCoBigM")
    if not verbose: M.setLogHandler(None)
    _fusion_set_time_limit(M, time_limit, is_mip=True)

    w  = M.variable("w",  n, mf.Domain.unbounded())
    Wd = M.variable("Wd", n, mf.Domain.greaterThan(0.0))
    b  = M.variable("b",  1, mf.Domain.unbounded())
    xi = M.variable("xi", m, mf.Domain.greaterThan(0.0))
    t  = M.variable("t",  n, mf.Domain.binary())

    M.constraint(mf.Expr.sum(t), mf.Domain.equalsTo(float(n - B)))

    one = mf.Expr.ones(n)
    M.constraint(mf.Expr.sub(w, mf.Expr.mul(Mbig, mf.Expr.sub(one, t))), mf.Domain.lessThan(0.0))
    M.constraint(mf.Expr.add(w, mf.Expr.mul(Mbig, mf.Expr.sub(one, t))), mf.Domain.greaterThan(0.0))

    half = mf.Expr.constTerm(1, 0.5)
    for j in range(n):
        M.constraint(mf.Expr.hstack(Wd.index(j), half, w.index(j)), mf.Domain.inRotatedQCone())

    A = mf.Matrix.dense(Phi_diag)
    sAw = mf.Expr.mul(A, w)
    for i in range(m):
        lhs = mf.Expr.mul(y[i], mf.Expr.add(sAw.index(i), b))
        M.constraint(mf.Expr.sub(lhs, mf.Expr.sub(1.0, xi.index(i))), mf.Domain.greaterThan(0.0))

    obj = mf.Expr.add(mf.Expr.mul(0.5, mf.Expr.sum(Wd)), mf.Expr.mul(C, mf.Expr.sum(xi)))
    M.objective(mf.ObjectiveSense.Minimize, obj)
    M.solve()

    objval = _fusion_obj_value(M)  # ★ 改这里

    sol = {
        "status": str(M.getPrimalSolutionStatus()),
        "t": np.array(t.level()).reshape(-1),
        "w": np.array(w.level()).reshape(-1),
        "obj": objval,                                   # ★ 改这里
    }
    M.dispose()
    return sol

# =========================================================
# CoP(K) (Gurobi) —— 指示约束，返回完整解 (w,b,xi,t)
# =========================================================
def solve_cop_kernel_gurobi(X: np.ndarray, y: np.ndarray, B: int, C: float,
                            kernel_set: Iterable[int], UB: Optional[float],
                            time_limit: float = 120.0, mipgap: Optional[float] = None,
                            verbose: bool = False, feature_names: Optional[List[str]] = None) -> Dict:
    import gurobipy as gp
    from gurobipy import GRB

    m, n = X.shape
    Phi_diag, Phi_cross_list, pairs = quad_map(X)
    P = len(pairs)

    model = gp.Model("CoP_K")
    model.Params.OutputFlag = 1 if verbose else 0
    model.Params.TimeLimit = time_limit
    if mipgap is not None:
        model.Params.MIPGap = mipgap
    model.Params.Seed = 0  # 可复现

    w_d  = model.addVars(n, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="w_d")
    w_c  = model.addVars(P, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="w_c")
    b    = model.addVar(lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="b")
    xi   = model.addVars(m, lb=0.0, vtype=GRB.CONTINUOUS, name="xi")
    t    = model.addVars(n, vtype=GRB.BINARY, name="t")  # 1=禁用

    # 指示约束：若 t_j=1，则 w_d[j]=0；交叉项同理
    for j in range(n):
        model.addGenConstrIndicator(t[j], 1, w_d[j], GRB.EQUAL, 0.0)
    for k, (i, j) in enumerate(pairs):
        model.addGenConstrIndicator(t[i], 1, w_c[k], GRB.EQUAL, 0.0)
        model.addGenConstrIndicator(t[j], 1, w_c[k], GRB.EQUAL, 0.0)

    # 预算
    model.addConstr(gp.quicksum(t[j] for j in range(n)) == n - B)

    # margin 逐样本
    for i in range(m):
        expr = b
        for j in range(n):
            expr += w_d[j] * float(Phi_diag[i, j])
        for k in range(P):
            expr += w_c[k] * float(Phi_cross_list[k][i])
        model.addConstr(float(y[i]) * expr >= 1.0 - xi[i])

    # (18) 至少从 kernel_set 里选一个
    K = sorted(set(int(j) for j in kernel_set))
    if K:
        model.addConstr(gp.quicksum(t[j] for j in K) <= len(K) - 1)

    # 固定核外特征禁用
    outside = sorted(set(range(n)) - set(K))
    for j in outside:
        model.addConstr(t[j] == 1)

    # 目标
    quad = gp.QuadExpr()
    for j in range(n): quad += w_d[j] * w_d[j]
    for k in range(P): quad += w_c[k] * w_c[k]
    obj = 0.5 * quad + C * gp.quicksum(xi[i] for i in range(m))
    model.setObjective(obj, GRB.MINIMIZE)
    if (UB is not None) and np.isfinite(UB):
        model.addQConstr(obj <= UB + 1e-9)

    model.optimize()

    sol = {
        "status": model.Status,
        "runtime": model.Runtime,
        "gap": model.MIPGap if model.SolCount > 0 else None,
        "obj": model.ObjVal if model.SolCount > 0 else None,
        "t": None,
        "selected_idx": [],
        "w_diag": None, "w_cross": None, "b": None, "xi": None
    }
    if model.SolCount > 0:
        tvec = np.array([t[j].X for j in range(n)])
        idx  = [j for j in range(n) if t[j].X < 0.5]  # t=0 selected
        w_dv = np.array([w_d[j].X for j in range(n)])
        w_cv = np.array([w_c[k].X for k in range(P)])
        bval = float(b.X)
        xiv  = np.array([xi[i].X for i in range(m)])
        sol.update(dict(t=tvec, selected_idx=idx, w_diag=w_dv, w_cross=w_cv, b=bval, xi=xiv))
        if feature_names:
            sol["selected_features"] = [feature_names[j] for j in idx]
    return sol



# =========================================================
# Algorithm 3: Heuristic procedure（全部使用 Algorithm 2）
# =========================================================
def heuristic_procedure_algo3(
    X: np.ndarray, y: np.ndarray, B: int, C: float,
    rho: int,
    ks_mosek_time: Optional[float] = 30.0,
    ks_mip_time: float = 60.0,
    ks_mip_gap: Optional[float] = 0.03,
    bound_time_per_j: Optional[float] = 10.0,
    l2_time: Optional[float] = 60.0,
    bigm_time: Optional[float] = 120.0,
    verbose: bool = True,
    feature_names: Optional[List[str]] = None
) -> Dict:
    # Step 2: KS#1
    ks1 = kernel_search(X, y, B, C, rho,
                        mosek_time=ks_mosek_time, mip_time=ks_mip_time, mip_gap=ks_mip_gap,
                        verbose=verbose, feature_names=feature_names)
    UB_hat = ks1["UB"]
    best   = ks1["best"]
    if verbose:
        print(f"[Alg3] KS#1 done: UB_hat={ 'inf' if not np.isfinite(UB_hat) else f'{UB_hat:.6g}' }, |S|={len(best['idx'])}")

    # 先把当前最优 UB 设成 KS#1 的 UB
    UB_star = UB_hat

    # ---------- NEW: 为 (19) 准备一个有限的 UB ----------
    l2_cached = None
    UB_for_bounds = UB_hat
    if not np.isfinite(UB_for_bounds):
        if verbose:
            print("[Alg3] KS 未得到有限 UB，尝试用 l2-NLSVM 生成 fallback UB...")
        try:
            l2_cached = solve_l2_nlsvm_mosek(X, y, C, time_limit=l2_time, verbose=verbose)
            UB_for_bounds = float(l2_cached["obj"])
            if verbose:
                print(f"[Alg3] Fallback UB from l2-NLSVM = {UB_for_bounds:.6g}")
            # 同时用它更新 UB_star（更好的上界）
            UB_star = UB_for_bounds
        except Exception as e:
            if verbose:
                print(f"[Alg3] l2-NLSVM 失败（{e}），将跳过 Big-M 步。")
            UB_for_bounds = np.nan  # 标记为不可用
    # ----------------------------------------------------

    # Step 4: 依据命题 4.1 计算 Big-M（仅当 UB 可用）
    M_val = None
    if np.isfinite(UB_for_bounds):
        M_val = compute_M_by_prop41(X, y, B, C, UB_for_bounds,
                                    time_per_bound=bound_time_per_j, verbose=verbose)
    else:
        if verbose:
            print("[Alg3] 无有限 UB，跳过 Big-M 边界计算与后续 Big-M → KS#2 流程。")
        # 直接输出 KS#1 的结果
        out = {
            "UB": UB_star,
            "w_diag": best["w_diag"], "w_cross": best["w_cross"], "b": best["b"], "xi": best["xi"],
            "selected_idx": best["idx"],
        }
        if feature_names:
            out["selected_features"] = [feature_names[j] for j in best["idx"]]
        return out

    # Step 5: 解 (1) l2-NLSVM（若前面已算，用缓存）
    if l2_cached is not None:
        l2 = l2_cached
    else:
        l2 = solve_l2_nlsvm_mosek(X, y, C, time_limit=l2_time, verbose=verbose)
    l1_over_B = float(np.sum(np.abs(np.r_[l2["w_diag"], l2["w_cross"]]))) / max(1, B)
    if verbose:
        print(f"[Alg3] l2-NLSVM obj={l2['obj']:.6g},  ||w||_1/B = {l1_over_B:.6g}  vs  M = {M_val:.6g}")

    # Step 6: 判断
    if (M_val is not None) and (M_val < l1_over_B):
        # Step 7: 用 Big-M 求一个起点
        bigm = solve_dshor_co_bigm_mosek(X, y, B, C, Mbig=M_val, time_limit=bigm_time, verbose=verbose)
        t0 = bigm["t"]
        init_sel = [j for j in range(X.shape[1]) if t0[j] < 0.5]
        if verbose:
            print(f"[Alg3] DShorCo-BigM done: |u*|={len(init_sel)}, obj={bigm['obj']:.6g}")

        # Step 8: KS#2
        ks2 = kernel_search(X, y, B, C, rho,
                            init_kernel=init_sel, init_UB=UB_star,
                            mosek_time=ks_mosek_time, mip_time=ks_mip_time, mip_gap=ks_mip_gap,
                            verbose=verbose, feature_names=feature_names)
        UB_bar = ks2["UB"]
        if verbose:
            print(f"[Alg3] KS#2 done: UB_bar={ 'inf' if not np.isfinite(UB_bar) else f'{UB_bar:.6g}' }, |S|={len(ks2['best']['idx'])}")

        if (UB_bar is not None) and np.isfinite(UB_bar) and (UB_bar <= UB_star):
            best = ks2["best"]
            UB_star = UB_bar
            if verbose:
                print(f"[Alg3] Accept KS#2 solution. New UB={UB_star:.6g}")
    else:
        if verbose:
            print("[Alg3] M ≥ ||w||_1/B，不触发 Big-M → KS#2；沿用 KS#1 解。")

    # 输出
    out = {
        "UB": UB_star,
        "w_diag": best["w_diag"], "w_cross": best["w_cross"], "b": best["b"], "xi": best["xi"],
        "selected_idx": best["idx"],
    }
    if feature_names:
        out["selected_features"] = [feature_names[j] for j in best["idx"]]
    return out



In [5]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum
from sklearn.metrics.pairwise import rbf_kernel
from ucimlrepo import fetch_ucirepo
import time

In [6]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
from sklearn.preprocessing import StandardScaler

breast_cancer = fetch_ucirepo(id=15)
X_df = breast_cancer.data.features.replace('?', np.nan).dropna()
y_series = breast_cancer.data.targets['Class'].loc[X_df.index]
feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)
y_int = pd.to_numeric(y_series, errors='raise').values
y = np.where(y_int == 2, 1.0, -1.0)  # 2→+1, 4→-1

B, C, rho = 5, 1.0, 4
res = kernel_search(
    X, y, B, C, rho,
    feature_names=feature_names,
    mosek_time=20.0,   # D-Shor-Co 限时
    mip_time=60.0,     # CoP(K) 限时
    mip_gap=0.03,
    verbose=True
)
print("UB:", res["UB"])
print("best obj:", res["best_obj"])
print("selected:", res.get("selected_features", res["selected_idx"]))



[D-Shor-Co] obj=87.9394  t_min..max=(0.444444, 0.444444)
[KS] n=9, rho=4  => N=3 groups (each ≤4)
[KS] iter 1/3  |K_k|=4  |K_bar|=0  |K_it|=7  UB=inf
Set parameter Username
Set parameter LicenseID to value 2629579
Academic license - for non-commercial use only - expires 2026-02-28
Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  60
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 687 rows, 738 columns and 32119 nonzeros
Model fingerprint: 0x1328cec4

In [ ]:
breast_cancer_wisconsin_prognostic = fetch_ucirepo(id=16)
X_df = breast_cancer_wisconsin_prognostic.data.features.replace('?', np.nan).dropna()
y_series = breast_cancer_wisconsin_prognostic.data.targets['Outcome'].loc[X_df.index]
feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# y_series: 形如包含 'N' / 'R' 的 pandas Series
y_str = y_series.astype(str).str.strip().str.upper().values

# 检查是否真的是二分类且只含 N/R
uniq = np.unique(y_str)
if not set(uniq).issubset({"N", "R"}) or len(uniq) != 2:
    raise ValueError(f"期望标签为 {{'N','R'}}，但实际唯一取值={uniq}")

# 约定：N -> +1，R -> -1  （和你之前的一致）
y = np.where(y_str == "N", 1.0, -1.0)

B, C, rho = 5, 1.0, 4
res = kernel_search(
    X, y, B, C, rho,
    feature_names=feature_names,
    mosek_time=20.0,   # D-Shor-Co 限时
    mip_time=60.0,     # CoP(K) 限时
    mip_gap=0.03,
    verbose=True
)
print("UB:", res["UB"])
print("best obj:", res["best_obj"])
print("selected:", res.get("selected_features", res["selected_idx"]))

[D-Shor-Co] obj=84.4431  t_min..max=(0, 1)
[KS] n=33, rho=4  => N=9 groups (each ≤4)
[KS] iter 1/9  |K_k|=4  |K_bar|=0  |K_it|=7  UB=inf
Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  60
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 222 rows, 789 columns and 109288 nonzeros
Model fingerprint: 0x4e68f2ba
Model has 561 quadratic objective terms
Model has 1089 simple general constraints
  1089 INDICATOR
Variable types: 756 continuous, 33 integer (

In [12]:
# =========================================================
# Co-NLSVM (Gurobi) vs Algorithm-3 Heuristic (same time budget)
#   - (13) D-Shor-Co ........... MOSEK Fusion (conic, diag-only)
#   - (19) bounds for w_j ...... MOSEK Fusion (conic, with safe UB)
#   - (1)  l2-NLSVM ............ MOSEK Fusion (conic, explicit quadratic map)
#   - CoP(K) ................... Gurobi (indicator constraints, with feasible MIP start)
# =========================================================
from __future__ import annotations
import time, math, itertools
import numpy as np
import pandas as pd
from typing import List, Dict, Optional, Iterable, Tuple

# -------------------- Helpers --------------------
def _fmt_num(x): 
    try:
        return "inf" if (x is None or not np.isfinite(x)) else f"{float(x):.6g}"
    except Exception:
        return str(x)

def _fusion_set_time_limit(M, seconds: float | None, is_mip: bool = False):
    """MOSEK Fusion time limit with version-agnostic params."""
    if seconds is None:
        return
    try:
        import mosek
        M.setSolverParam(mosek.dparam.optimizer_max_time, float(seconds))
        if is_mip:
            M.setSolverParam(mosek.dparam.mio_max_time, float(seconds))
    except Exception:
        # textual fallbacks
        try:
            M.setSolverParam("optimizerMaxTime", float(seconds))
        except Exception:
            pass
        if is_mip:
            try:
                M.setSolverParam("mioMaxTime", float(seconds))
            except Exception:
                pass

def _fusion_obj_value(M):
    """Read objective value in a version-compatible way."""
    try:
        return float(M.primalObjValue())
    except Exception:
        return float(M.objectiveValue())

def quad_map(X: np.ndarray):
    """
    显式二次映射：
      phi(x) = [ x_1^2, ..., x_n^2, sqrt(2)*x_1 x_2, ..., sqrt(2)*x_{n-1} x_n ]
    返回：Phi_diag (m×n), Phi_cross_list (长度 P 的 m 向量列表), pairs (索引对)
    """
    m, n = X.shape
    Phi_diag = X**2
    pairs = list(itertools.combinations(range(n), 2))
    sqrt2 = np.sqrt(2.0)
    Phi_cross_list = [sqrt2*(X[:, i]*X[:, j]) for (i, j) in pairs]
    return Phi_diag, Phi_cross_list, pairs

def _ensure_kernel_size(order: Iterable[int], K_it: List[int], B: int, extra: int = 0) -> List[int]:
    """确保 |K_it| ≥ B（可加冗余 extra），否则按 D-Shor-Co 排名补足。"""
    want = max(B + extra, len(K_it))
    Kset = set(int(j) for j in K_it)
    for idx in list(order):
        if len(Kset) >= want: break
        if int(idx) not in Kset: Kset.add(int(idx))
    return sorted(Kset)

# -------------------- (13) D-Shor-Co (MOSEK, diag-only) --------------------
def solve_dshor_co_mosek(X: np.ndarray, y: np.ndarray, B: int, C: float,
                         time_limit: float | None = None, verbose=False) -> Dict:
    import mosek.fusion as mf
    m, n = X.shape
    Phi_diag, _, _ = quad_map(X)
    M = mf.Model("DShorCo")
    if not verbose: M.setLogHandler(None)
    _fusion_set_time_limit(M, time_limit, is_mip=False)

    # variables
    w  = M.variable("w",  n, mf.Domain.unbounded())
    Wd = M.variable("Wd", n, mf.Domain.greaterThan(0.0))
    b  = M.variable("b",  1, mf.Domain.unbounded())
    xi = M.variable("xi", m, mf.Domain.greaterThan(0.0))
    t  = M.variable("t",  n, mf.Domain.inRange(0.0, 1.0))

    # sum t = n - B
    M.constraint(mf.Expr.sum(t), mf.Domain.equalsTo(float(n - B)))

    # rotated cone: w_j^2 <= Wd_j
    half = mf.Expr.constTerm(1, 0.5)
    for j in range(n):
        M.constraint(mf.Expr.hstack(Wd.index(j), half, w.index(j)), mf.Domain.inRotatedQCone())

    # margin constraints (diag-only)
    A = mf.Matrix.dense(Phi_diag)
    sAw = mf.Expr.mul(A, w)
    for i in range(m):
        lhs = mf.Expr.mul(float(y[i]), mf.Expr.add(sAw.index(i), b))
        M.constraint(mf.Expr.sub(lhs, mf.Expr.sub(1.0, xi.index(i))), mf.Domain.greaterThan(0.0))

    obj = mf.Expr.add(mf.Expr.mul(0.5, mf.Expr.sum(Wd)), mf.Expr.mul(C, mf.Expr.sum(xi)))
    M.objective(mf.ObjectiveSense.Minimize, obj)
    M.solve()

    val = _fusion_obj_value(M)
    out = {"obj": val, "t_rank": np.array(t.level()).reshape(-1)}
    M.dispose()
    return out

# -------------------- CoP(K) with Gurobi (indicator, feasible start) --------------------
def solve_cop_kernel_gurobi(X: np.ndarray, y: np.ndarray, B: int, C: float,
                            kernel_set: Iterable[int], UB: float | None,
                            time_limit: float, mipgap: float | None,
                            verbose=False, feature_names: List[str] | None = None,
                            start_idx: List[int] | None = None,
                            favor_feasible: bool=True) -> Dict:
    import gurobipy as gp
    from gurobipy import GRB

    m, n = X.shape
    Phi_diag, Phi_cross_list, pairs = quad_map(X)
    P = len(pairs)

    model = gp.Model("CoP_K")
    model.Params.OutputFlag = 1 if verbose else 0
    model.Params.TimeLimit  = time_limit
    if mipgap is not None: model.Params.MIPGap = mipgap
    if favor_feasible:
        model.Params.MIPFocus   = 1
        model.Params.Heuristics = 0.5
        model.Params.Presolve   = 2
        model.Params.Cuts       = 1
    model.Params.Seed = 0

    w_d  = model.addVars(n, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="w_d")
    w_c  = model.addVars(P, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="w_c")
    b    = model.addVar(lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="b")
    xi   = model.addVars(m, lb=0.0, vtype=GRB.CONTINUOUS, name="xi")
    t    = model.addVars(n, vtype=GRB.BINARY, name="t")  # 1 = 禁用

    # indicators
    for j in range(n):
        t[j].BranchPriority = 10
        model.addGenConstrIndicator(t[j], 1, w_d[j], GRB.EQUAL, 0.0)
    for k, (i, j) in enumerate(pairs):
        model.addGenConstrIndicator(t[i], 1, w_c[k], GRB.EQUAL, 0.0)
        model.addGenConstrIndicator(t[j], 1, w_c[k], GRB.EQUAL, 0.0)

    # budget: exactly B selected
    model.addConstr(gp.quicksum(t[j] for j in range(n)) == n - B, name="budget")

    # margins
    for i in range(m):
        expr = b
        for j in range(n): expr += w_d[j]*float(Phi_diag[i, j])
        for k in range(P): expr += w_c[k]*float(Phi_cross_list[k][i])
        model.addConstr(float(y[i])*expr >= 1.0 - xi[i], name=f"margin_{i}")

    # kernel constraints
    K = sorted(set(int(j) for j in kernel_set))
    if K:
        model.addConstr(gp.quicksum(t[j] for j in K) <= len(K) - 1, name="atleast_one_in_kernel")
    outside = sorted(set(range(n)) - set(K))
    for j in outside: model.addConstr(t[j] == 1, name=f"fix_outside_{j}")

    # objective and optional UB cap (17)
    quad = gp.QuadExpr()
    for j in range(n): quad += w_d[j]*w_d[j]
    for k in range(P): quad += w_c[k]*w_c[k]
    obj = 0.5*quad + C*gp.quicksum(xi[i] for i in range(m))
    model.setObjective(obj, GRB.MINIMIZE)
    if (UB is not None) and np.isfinite(UB): model.addQConstr(obj <= UB + 1e-9, name="obj_ub")

    # feasible MIP Start
    for j in range(n): t[j].Start = 1
    seed = (start_idx if start_idx else K)[:min(B, len(K))]
    for j in seed: t[j].Start = 0
    b.Start = 0.0
    for j in range(n): w_d[j].Start = 0.0
    for k in range(P): w_c[k].Start = 0.0
    for i in range(m): xi[i].Start = 1.0

    t0 = time.time()
    model.optimize()
    rt = time.time() - t0

    sol = {
        "status": model.Status,
        "runtime": rt,
        "objval": model.ObjVal if model.SolCount>0 else np.inf,
        "gap":    model.MIPGap if model.SolCount>0 else np.inf,
        "selected_idx": [],
        "selected_features": [],
    }
    if model.SolCount>0:
        idx = [j for j in range(n) if t[j].X < 0.5]
        sol["selected_idx"] = idx
        if feature_names:
            sol["selected_features"] = [feature_names[j] for j in idx]
    return sol

# -------------------- (1) l2-NLSVM (MOSEK, conic) --------------------
def solve_l2_nlsvm_mosek(X: np.ndarray, y: np.ndarray, C: float,
                         time_limit: float | None, verbose=False) -> Dict:
    """
    用旋转二阶锥线性化 ||w||^2，避免 Expr.dot(Variable,Variable)。
    """
    import mosek.fusion as mf
    m, n = X.shape
    Phi_diag, Phi_cross_list, pairs = quad_map(X); P = len(pairs)
    Phi_cross = np.vstack([pc for pc in Phi_cross_list]).T if P>0 else np.zeros((m,0))

    M = mf.Model("l2-NLSVM")
    if not verbose: M.setLogHandler(None)
    _fusion_set_time_limit(M, time_limit, is_mip=False)

    # variables
    w_d = M.variable("w_d", n, mf.Domain.unbounded())
    w_c = M.variable("w_c", P, mf.Domain.unbounded()) if P>0 else None
    b   = M.variable("b", 1, mf.Domain.unbounded())
    xi  = M.variable("xi", m, mf.Domain.greaterThan(0.0))

    # auxiliaries for rotated cones
    Wd  = M.variable("Wd", n, mf.Domain.greaterThan(0.0))
    Wc  = M.variable("Wc", P, mf.Domain.greaterThan(0.0)) if P>0 else None

    half = mf.Expr.constTerm(1, 0.5)
    for j in range(n):
        M.constraint(mf.Expr.hstack(Wd.index(j), half, w_d.index(j)), mf.Domain.inRotatedQCone())
    if P>0:
        for k in range(P):
            M.constraint(mf.Expr.hstack(Wc.index(k), half, w_c.index(k)), mf.Domain.inRotatedQCone())

    # margins
    A_d = mf.Matrix.dense(Phi_diag); s = mf.Expr.mul(A_d, w_d)
    if P>0:
        A_c = mf.Matrix.dense(Phi_cross); s = mf.Expr.add(s, mf.Expr.mul(A_c, w_c))
    for i in range(m):
        lhs = mf.Expr.mul(float(y[i]), mf.Expr.add(s.index(i), b))
        M.constraint(mf.Expr.sub(lhs, mf.Expr.sub(1.0, xi.index(i))), mf.Domain.greaterThan(0.0))

    # objective: 0.5*(sum Wd + sum Wc) + C*sum xi
    obj = mf.Expr.add(mf.Expr.mul(0.5, mf.Expr.sum(Wd)), mf.Expr.mul(C, mf.Expr.sum(xi)))
    if P>0:
        obj = mf.Expr.add(obj, mf.Expr.mul(0.5, mf.Expr.sum(Wc)))
    M.objective(mf.ObjectiveSense.Minimize, obj)
    M.solve()

    w_diag  = np.array(w_d.level()).reshape(-1)
    w_cross = np.array(w_c.level()).reshape(-1) if P>0 else np.zeros(P)
    bval    = float(b.level()[0])
    xi_v    = np.array(xi.level()).reshape(-1)
    objval  = _fusion_obj_value(M)

    M.dispose()
    return {"obj": objval, "w_diag": w_diag, "w_cross": w_cross, "b": bval, "xi": xi_v}

# -------------------- (19) bounds with safe fallback --------------------
def bound_wj_mosek(X: np.ndarray, y: np.ndarray, B: int, C: float, UB: float,
                   j: int, sense: str, time_limit: float | None) -> Tuple[float, bool]:
    """
    返回 (值, is_optimal)。若不可行/非最优/异常，由外层给兜底界。
    """
    import mosek.fusion as mf
    m, n = X.shape
    Phi_diag, _, _ = quad_map(X)

    M = mf.Model(f"Bound_w{j}_{sense}")
    _fusion_set_time_limit(M, time_limit, is_mip=False)

    w  = M.variable("w",  n, mf.Domain.unbounded())
    Wd = M.variable("Wd", n, mf.Domain.greaterThan(0.0))
    b  = M.variable("b",  1, mf.Domain.unbounded())
    xi = M.variable("xi", m, mf.Domain.greaterThan(0.0))
    t  = M.variable("t",  n, mf.Domain.inRange(0.0, 1.0))

    M.constraint(mf.Expr.sum(t), mf.Domain.equalsTo(float(n - B)))

    half = mf.Expr.constTerm(1, 0.5)
    for jj in range(n):
        M.constraint(mf.Expr.hstack(Wd.index(jj), half, w.index(jj)), mf.Domain.inRotatedQCone())

    A = mf.Matrix.dense(Phi_diag); sAw = mf.Expr.mul(A, w)
    for i in range(m):
        lhs = mf.Expr.mul(float(y[i]), mf.Expr.add(sAw.index(i), b))
        M.constraint(mf.Expr.sub(lhs, mf.Expr.sub(1.0, xi.index(i))), mf.Domain.greaterThan(0.0))

    if not np.isfinite(UB):
        UB = 1e6
    M.constraint(
        mf.Expr.add(mf.Expr.mul(0.5, mf.Expr.sum(Wd)), mf.Expr.mul(C, mf.Expr.sum(xi))),
        mf.Domain.lessThan(UB + 1e-9)
    )

    M.objective(mf.ObjectiveSense.Minimize if sense=="min" else mf.ObjectiveSense.Maximize, w.index(j))
    try:
        M.solve()
        ps = M.getPrimalSolutionStatus()
        import mosek.fusion as mf
        ok = ps in (mf.SolutionStatus.Optimal, mf.SolutionStatus.NearOptimal)
        val = float(w.level()[j]) if ok else np.nan
    except Exception:
        ok, val = False, np.nan
    finally:
        M.dispose()
    return val, ok

def compute_M_by_prop41(X: np.ndarray, y: np.ndarray, B: int, C: float, UB: float,
                        time_per_bound: float | None, verbose=False) -> float:
    n = X.shape[1]
    safe_cap = float(np.sqrt(max(0.0, 2.0 * UB))) if np.isfinite(UB) else 1e6
    Mvals = []
    for j in range(n):
        lo, ok_lo = bound_wj_mosek(X, y, B, C, UB, j, "min", time_per_bound)
        hi, ok_hi = bound_wj_mosek(X, y, B, C, UB, j, "max", time_per_bound)
        if not (ok_lo and np.isfinite(lo)): lo = -safe_cap
        if not (ok_hi and np.isfinite(hi)): hi =  safe_cap
        Mj = max(abs(lo), abs(hi))
        Mvals.append(Mj)
        if verbose:
            print(f"[Prop4.1] j={j} lo={lo:.6g} hi={hi:.6g} |.|max={Mj:.6g} "
                  f"{'' if (ok_lo and ok_hi) else '(fallback)'}")
    M = float(np.max(Mvals))
    if verbose: print(f"[Prop4.1] M = {M:.6g}")
    return M

# -------------------- (14) D-Shor-Co-BigM (MOSEK, diag-only MISOCP) --------------------
def solve_dshor_co_bigm_mosek(X: np.ndarray, y: np.ndarray, B: int, C: float, Mbig: float,
                              time_limit: float | None, verbose=False) -> Dict:
    import mosek.fusion as mf
    m, n = X.shape
    Phi_diag, _, _ = quad_map(X)
    M = mf.Model("DShorCoBigM")
    if not verbose: M.setLogHandler(None)
    _fusion_set_time_limit(M, time_limit, is_mip=True)

    w  = M.variable("w",  n, mf.Domain.unbounded())
    Wd = M.variable("Wd", n, mf.Domain.greaterThan(0.0))
    b  = M.variable("b",  1, mf.Domain.unbounded())
    xi = M.variable("xi", m, mf.Domain.greaterThan(0.0))
    t  = M.variable("t",  n, mf.Domain.binary())

    M.constraint(mf.Expr.sum(t), mf.Domain.equalsTo(float(n - B)))

    one = mf.Expr.ones(n)
    M.constraint(mf.Expr.sub(w, mf.Expr.mul(Mbig, mf.Expr.sub(one, t))), mf.Domain.lessThan(0.0))
    M.constraint(mf.Expr.add(w, mf.Expr.mul(Mbig, mf.Expr.sub(one, t))), mf.Domain.greaterThan(0.0))

    half = mf.Expr.constTerm(1, 0.5)
    for j in range(n):
        M.constraint(mf.Expr.hstack(Wd.index(j), half, w.index(j)), mf.Domain.inRotatedQCone())

    A = mf.Matrix.dense(Phi_diag); sAw = mf.Expr.mul(A, w)
    for i in range(m):
        lhs = mf.Expr.mul(float(y[i]), mf.Expr.add(sAw.index(i), b))
        M.constraint(mf.Expr.sub(lhs, mf.Expr.sub(1.0, xi.index(i))), mf.Domain.greaterThan(0.0))

    obj = mf.Expr.add(mf.Expr.mul(0.5, mf.Expr.sum(Wd)), mf.Expr.mul(C, mf.Expr.sum(xi)))
    M.objective(mf.ObjectiveSense.Minimize, obj)
    M.solve()

    objv = _fusion_obj_value(M)
    tvec = np.array(t.level()).reshape(-1)
    M.dispose()
    return {"obj": objv, "t": tvec}

# -------------------- KS with global budget --------------------
def kernel_search_budgeted(
    X: np.ndarray, y: np.ndarray, B: int, C: float, rho: int,
    feature_names: List[str] | None,
    dshor_time: float, ks_total_time: float, mip_gap: float | None,
    verbose: bool = True
) -> Dict:
    start = time.monotonic()
    rel = solve_dshor_co_mosek(X, y, B, C, time_limit=dshor_time, verbose=verbose)
    order = np.argsort(rel["t_rank"])
    m, n = X.shape
    rho = max(1, min(rho, n))
    groups = [order[i*rho: min((i+1)*rho, n)].tolist() for i in range(math.ceil(n/rho))]

    K_bar: List[int] = []
    UB, best_obj, best_gap = np.inf, np.inf, np.inf
    best_idx: List[int] = []
    logs = []

    for k, K_k in enumerate(groups, start=1):
        elapsed = time.monotonic() - start
        remain  = max(0.0, ks_total_time - elapsed)
        if remain <= 1e-3: break
        per_iter = max(1.0, remain / (len(groups) - (k-1)))

        K_it_raw = sorted(set(K_k) | set(K_bar))
        K_it = _ensure_kernel_size(order, K_it_raw, B, extra=max(1, B//2))
        seed = best_idx if best_idx else K_it[:B]

        sol = solve_cop_kernel_gurobi(
            X, y, B, C,
            kernel_set=K_it, UB=UB,
            time_limit=per_iter, mipgap=mip_gap, verbose=verbose,
            feature_names=feature_names, start_idx=seed, favor_feasible=True
        )
        if np.isfinite(sol["objval"]) and (sol["objval"] < best_obj):
            best_obj, best_gap = sol["objval"], sol["gap"]
            best_idx = sol["selected_idx"]
        if np.isfinite(sol["objval"]) and (sol["objval"] < UB): UB = sol["objval"]

        sel_set = set(sol["selected_idx"])
        K_plus  = [j for j in K_k if j in sel_set]
        K_bar   = sorted(set(K_bar).union(K_plus))

        logs.append({"k":k, "obj":sol["objval"], "gap":sol["gap"], "rt":sol["runtime"], "K_plus":len(K_plus)})

        if verbose:
            print(f"[KS] iter {k}/{len(groups)} -> obj={_fmt_num(sol['objval'])}, gap={_fmt_num(sol['gap'])}, K^+={len(K_plus)}")

    return {"UB": UB, "best_obj": best_obj, "best_gap": best_gap,
            "selected_idx": best_idx, "logs": logs}

# -------------------- Algorithm-3 with budget --------------------
def algorithm3_budgeted(
    X: np.ndarray, y: np.ndarray, B: int, C: float, rho: int,
    total_time: float, feature_names: List[str] | None = None, verbose=True
) -> Dict:
    """
    时间切片（可调整）：
      T = total_time
      KS#1: dshor 10% T + ks 40% T
      bounds+l2+bigM: 30% T
      KS#2: 20% T
    """
    T = float(total_time)
    t_dshor1 = 0.10*T; t_ks1 = 0.40*T
    t_bounds_l2_bigm = 0.30*T
    t_ks2 = 0.20*T

    # KS#1
    ks1 = kernel_search_budgeted(X, y, B, C, rho, feature_names, dshor_time=t_dshor1,
                                 ks_total_time=t_ks1, mip_gap=0.03, verbose=verbose)
    UB_star, idx_star, gap_star = ks1["UB"], ks1["selected_idx"], ks1["best_gap"]
    if verbose: print(f"[Alg3] KS#1: UB={_fmt_num(UB_star)}, gap={_fmt_num(gap_star)}")

    if not np.isfinite(UB_star):
        # 极少情况：直接返回 KS#1 结果
        return {"UB": np.inf, "gap": np.inf, "runtime": t_dshor1 + t_ks1, "selected_idx": []}

    # bounds + l2 + bigM
    start = time.monotonic()

    # l2-NLSVM 作为参考 UB
    l2 = solve_l2_nlsvm_mosek(X, y, C, time_limit=max(1.0, 0.3*t_bounds_l2_bigm), verbose=False)
    l2_obj = float(l2["obj"])

    # D-Shor-Co 的最优值，也用于保证 (19) 可行
    rel_chk = solve_dshor_co_mosek(X, y, B, C, time_limit=max(1.0, 0.2*t_bounds_l2_bigm), verbose=False)
    obj_dshor = float(rel_chk["obj"])

    # (19) 的 UB 取三者的最大值（确保可行）
    UB_for_bounds = max(
        (UB_star if np.isfinite(UB_star) else -np.inf),
        l2_obj, obj_dshor
    ) + 1e-9

    # 分配每个 j 的 min/max 界时间
    remain = max(0.0, t_bounds_l2_bigm - (time.monotonic()-start))
    per_bound = max(0.5, remain / max(1, 2*X.shape[1]))
    Mval = compute_M_by_prop41(X, y, B, C, UB_for_bounds, time_per_bound=per_bound, verbose=False)

    # BigM 触发判断
    l1_over_B = float(np.sum(np.abs(np.r_[l2["w_diag"], l2["w_cross"]]))) / max(1, B)
    if verbose: print(f"[Alg3] M={_fmt_num(Mval)}, ||w||_1/B={_fmt_num(l1_over_B)}")
    do_ks2 = (np.isfinite(Mval) and (Mval < l1_over_B))

    # BigM warm-start → 初始化选择
    if do_ks2:
        remain = max(0.0, t_bounds_l2_bigm - (time.monotonic()-start))
        bigm = solve_dshor_co_bigm_mosek(X, y, B, C, Mbig=Mval, time_limit=max(1.0, 0.5*remain), verbose=False)
        init_sel = [j for j in range(X.shape[1]) if bigm["t"][j] < 0.5]
    else:
        init_sel = idx_star

    # KS#2（使用 init_sel 作种子）
    if t_ks2 > 1.0:
        rel2 = solve_dshor_co_mosek(X, y, B, C, time_limit=0.2*t_ks2, verbose=False)
        order = np.argsort(rel2["t_rank"])
        m, n = X.shape
        rho2 = max(1, min(rho, n))
        groups = [order[i*rho2: min((i+1)*rho2, n)].tolist() for i in range(math.ceil(n/rho2))]
        if groups:
            groups[0] = sorted(set(groups[0]).union(init_sel))

        start2 = time.monotonic()
        UB2, best_obj2, best_gap2 = UB_star, UB_star, gap_star
        best_idx2: List[int] = idx_star
        K_bar: List[int] = list(init_sel)
        for k, K_k in enumerate(groups, start=1):
            elapsed = time.monotonic() - start2
            remain = max(0.0, t_ks2 - elapsed)
            if remain <= 1e-3: break
            per_iter = max(1.0, remain/(len(groups)-(k-1)))

            K_it_raw = sorted(set(K_k) | set(K_bar))
            K_it = _ensure_kernel_size(order, K_it_raw, B, extra=max(1, B//2))
            seed = best_idx2 if best_idx2 else K_it[:B]

            sol = solve_cop_kernel_gurobi(
                X, y, B, C, kernel_set=K_it, UB=UB2,
                time_limit=per_iter, mipgap=0.03, verbose=verbose,
                feature_names=feature_names, start_idx=seed, favor_feasible=True
            )
            if np.isfinite(sol["objval"]) and (sol["objval"] < best_obj2):
                best_obj2, best_gap2, best_idx2 = sol["objval"], sol["gap"], sol["selected_idx"]
            if np.isfinite(sol["objval"]) and (sol["objval"] < UB2): UB2 = sol["objval"]

            K_plus = [j for j in K_k if j in set(sol["selected_idx"])]
            K_bar  = sorted(set(K_bar).union(K_plus))

            if verbose:
                print(f"[KS2] iter {k}/{len(groups)} -> obj={_fmt_num(sol['objval'])}, gap={_fmt_num(sol['gap'])}, K^+={len(K_plus)}")

        if np.isfinite(best_obj2) and (best_obj2 <= UB_star):
            UB_star, gap_star, idx_star = best_obj2, best_gap2, best_idx2
            if verbose: print(f"[Alg3] Accept KS#2: UB={_fmt_num(UB_star)}, gap={_fmt_num(gap_star)}")

    return {"UB": UB_star, "gap": gap_star, "runtime": T, "selected_idx": idx_star}

# -------------------- Full Co-NLSVM (same total time) --------------------
def conlsvm_full_same_budget(X: np.ndarray, y: np.ndarray, B: int, C: float,
                             total_time: float, feature_names: List[str] | None = None,
                             verbose=False) -> Dict:
    # 用 D-Shor-Co 的排序前 B 作为 warm-start 种子
    rel = solve_dshor_co_mosek(X, y, B, C, time_limit=0.1*total_time, verbose=False)
    order = np.argsort(rel["t_rank"]).tolist()
    seed = order[:min(B, len(order))]
    sol = solve_cop_kernel_gurobi(
        X, y, B, C, kernel_set=range(X.shape[1]), UB=None,
        time_limit=0.9*total_time, mipgap=0.03, verbose=verbose,
        feature_names=feature_names, start_idx=seed, favor_feasible=True
    )
    return {"UB": sol["objval"], "gap": sol["gap"], "runtime": 0.1*total_time + sol["runtime"],
            "selected_idx": sol["selected_idx"]}

# -------------------- One-call comparison --------------------
def run_comparison(X: np.ndarray, y: np.ndarray, B: int, C: float,
                   total_time: float, rho: int = 4, feature_names: List[str] | None = None,
                   verbose=True) -> pd.DataFrame:
    # Heuristic Algorithm-3
    res_alg3 = algorithm3_budgeted(X, y, B, C, rho, total_time, feature_names, verbose=verbose)
    # Full Co-NLSVM
    res_full = conlsvm_full_same_budget(X, y, B, C, total_time, feature_names, verbose=verbose)

    rows = [
        {"method":"Algorithm-3 (KS+Bounds+BigM+KS)", "UB":res_alg3["UB"], "MIPGap":res_alg3["gap"], "runtime":res_alg3["runtime"],
         "selected_k":len(res_alg3["selected_idx"])},
        {"method":"Full Co-NLSVM (indicator)",       "UB":res_full["UB"], "MIPGap":res_full["gap"], "runtime":res_full["runtime"],
         "selected_k":len(res_full["selected_idx"])},
    ]
    df = pd.DataFrame(rows)
    print("\n=== Comparison (same total time) ===")
    print(df.to_string(index=False, formatters={"UB":_fmt_num, "MIPGap":_fmt_num, "runtime":lambda x:f"{x:.2f}s"}))
    return df


In [13]:
breast_cancer_wisconsin_prognostic = fetch_ucirepo(id=16)
X_df = breast_cancer_wisconsin_prognostic.data.features.replace('?', np.nan).dropna()
y_series = breast_cancer_wisconsin_prognostic.data.targets['Outcome'].loc[X_df.index]
feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# y_series: 形如包含 'N' / 'R' 的 pandas Series
y_str = y_series.astype(str).str.strip().str.upper().values

# 检查是否真的是二分类且只含 N/R
uniq = np.unique(y_str)
if not set(uniq).issubset({"N", "R"}) or len(uniq) != 2:
    raise ValueError(f"期望标签为 {{'N','R'}}，但实际唯一取值={uniq}")

# 约定：N -> +1，R -> -1  （和你之前的一致）
y = np.where(y_str == "N", 1.0, -1.0)

# 假设你已准备好 X, y（y ∈ {-1,+1}）, 以及 feature_names（可选）
B, C = 5, 1.0
T_total = 120.0   # 两侧统一总时间（秒）
rho = 4

df_cmp = run_comparison(X, y, B, C, total_time=T_total, rho=rho, feature_names=feature_names, verbose=True)


Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 5.2766272555551650e+00
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  5.2766272555551650e+00
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 222 rows, 789 columns and 109288 nonzeros
Model fingerprint: 0x4e68f2ba
Model has 561 quadratic objective terms
Model has 1089 simple general constraints
  1089 INDICATOR
Variable types: 756 continuous, 33 integer (33 binary)
Coefficient statistics:
  Matrix range     [5e-08, 7e+01]
  Objective range  [1e+00, 1

In [22]:
breast_cancer_wisconsin_prognostic = fetch_ucirepo(id=16)
X_df = breast_cancer_wisconsin_prognostic.data.features.replace('?', np.nan).dropna()
y_series = breast_cancer_wisconsin_prognostic.data.targets['Outcome'].loc[X_df.index]
feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# y_series: 形如包含 'N' / 'R' 的 pandas Series
y_str = y_series.astype(str).str.strip().str.upper().values

# 检查是否真的是二分类且只含 N/R
uniq = np.unique(y_str)
if not set(uniq).issubset({"N", "R"}) or len(uniq) != 2:
    raise ValueError(f"期望标签为 {{'N','R'}}，但实际唯一取值={uniq}")

# 约定：N -> +1，R -> -1  （和你之前的一致）
y = np.where(y_str == "N", 1.0, -1.0)

# 假设你已准备好 X, y（y ∈ {-1,+1}）, 以及 feature_names（可选）
B, C = 10, 1.0
T_total = 120.0   # 两侧统一总时间（秒）
rho = 4

df_cmp = run_comparison(X, y, B, C, total_time=T_total, rho=rho, feature_names=feature_names, verbose=True)


Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 5.2963070000001204e+00
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  5.2963070000001204e+00
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 214 rows, 789 columns and 109288 nonzeros
Model fingerprint: 0x2c27b0ac
Model has 561 quadratic objective terms
Model has 1089 simple general constraints
  1089 INDICATOR
Variable types: 756 continuous, 33 integer (33 binary)
Coefficient statistics:
  Matrix range     [5e-08, 7e+01]
  Objective range  [1e+00, 1

In [ ]:
# 读取 UCI 数据
ionosphere = fetch_ucirepo(id=52)  
X_df = ionosphere.data.features
y_series = ionosphere.data.targets['Class']

# 清洗与对齐
X_df = X_df.replace('?', np.nan).dropna()
y_series = y_series.loc[X_df.index]

feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# y_series: 形如包含 'N' / 'R' 的 pandas Series
y_str = y_series.astype(str).str.strip().str.upper().values

# 检查是否真的是二分类且只含 g/b
uniq = np.unique(y_str)
if not set(uniq).issubset({"G", "B"}) or len(uniq) != 2:
    raise ValueError(f"期望标签为 {{'G','B'}}，但实际唯一取值={uniq}")

# 约定：g -> +1，b -> -1  （和你之前的一致）
y = np.where(y_str == "G", 1.0, -1.0)

# 假设你已准备好 X, y（y ∈ {-1,+1}）, 以及 feature_names（可选）
B, C = 5, 1.0
T_total = 120.0   # 两侧统一总时间（秒）
rho = 4

df_cmp = run_comparison(X, y, B, C, total_time=T_total, rho=rho, feature_names=feature_names, verbose=True)


Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 5.2520095999999388e+00
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  5.2520095999999388e+00
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 380 rows, 981 columns and 197681 nonzeros
Model fingerprint: 0xf22b23ae
Model has 595 quadratic objective terms
Model has 1156 simple general constraints
  1156 INDICATOR
Variable types: 947 continuous, 34 integer (34 binary)
Coefficient statistics:
  Matrix range     [6e-08, 1e+01]
  Objective range  [1e+00, 1

In [20]:
# 读取 UCI 数据
ionosphere = fetch_ucirepo(id=52)  
X_df = ionosphere.data.features
y_series = ionosphere.data.targets['Class']

# 清洗与对齐
X_df = X_df.replace('?', np.nan).dropna()
y_series = y_series.loc[X_df.index]

feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# y_series: 形如包含 'N' / 'R' 的 pandas Series
y_str = y_series.astype(str).str.strip().str.upper().values

# 检查是否真的是二分类且只含 g/b
uniq = np.unique(y_str)
if not set(uniq).issubset({"G", "B"}) or len(uniq) != 2:
    raise ValueError(f"期望标签为 {{'G','B'}}，但实际唯一取值={uniq}")

# 约定：g -> +1，b -> -1  （和你之前的一致）
y = np.where(y_str == "G", 1.0, -1.0)

# 假设你已准备好 X, y（y ∈ {-1,+1}）, 以及 feature_names（可选）
B, C = 10, 1.0
T_total = 120.0   # 两侧统一总时间（秒）
rho = 4

df_cmp = run_comparison(X, y, B, C, total_time=T_total, rho=rho, feature_names=feature_names, verbose=True)


Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 5.2753340222221192e+00
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  5.2753340222221192e+00
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 372 rows, 981 columns and 197681 nonzeros
Model fingerprint: 0xdfcdb61a
Model has 595 quadratic objective terms
Model has 1156 simple general constraints
  1156 INDICATOR
Variable types: 947 continuous, 34 integer (34 binary)
Coefficient statistics:
  Matrix range     [6e-08, 1e+01]
  Objective range  [1e+00, 1

In [17]:
# 读取 UCI 数据
parkinsons = fetch_ucirepo(id=174) 

X_df = parkinsons.data.features
y_series = parkinsons.data.targets['status']

# 清洗与对齐
X_df = X_df.replace('?', np.nan).dropna()
y_series = y_series.loc[X_df.index]

feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# 正确的标签编码（0/1 是整数）
y_int = pd.to_numeric(y_series, errors='raise').values
uniq = np.unique(y_int)
assert set(uniq) == {0, 1}, f"意外的类别：{uniq}"
y = np.where(y_int == 0, 1.0, -1.0)  # 0=+1, 1=-1

# 假设你已准备好 X, y（y ∈ {-1,+1}）, 以及 feature_names（可选）
B, C = 5, 1.0
T_total = 120.0   # 两侧统一总时间（秒）
rho = 4

df_cmp = run_comparison(X, y, B, C, total_time=T_total, rho=rho, feature_names=feature_names, verbose=True)


Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 7.9267643166664739e+00
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  7.9267643166664739e+00
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 212 rows, 471 columns and 49769 nonzeros
Model fingerprint: 0xa69c3314
Model has 253 quadratic objective terms
Model has 484 simple general constraints
  484 INDICATOR
Variable types: 449 continuous, 22 integer (22 binary)
Coefficient statistics:
  Matrix range     [5e-09, 6e+01]
  Objective range  [1e+00, 1e+0

In [21]:
# 读取 UCI 数据
parkinsons = fetch_ucirepo(id=174) 

X_df = parkinsons.data.features
y_series = parkinsons.data.targets['status']

# 清洗与对齐
X_df = X_df.replace('?', np.nan).dropna()
y_series = y_series.loc[X_df.index]

feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# 正确的标签编码（0/1 是整数）
y_int = pd.to_numeric(y_series, errors='raise').values
uniq = np.unique(y_int)
assert set(uniq) == {0, 1}, f"意外的类别：{uniq}"
y = np.where(y_int == 0, 1.0, -1.0)  # 0=+1, 1=-1

# 假设你已准备好 X, y（y ∈ {-1,+1}）, 以及 feature_names（可选）
B, C = 10, 1.0
T_total = 120.0   # 两侧统一总时间（秒）
rho = 4

df_cmp = run_comparison(X, y, B, C, total_time=T_total, rho=rho, feature_names=feature_names, verbose=True)


Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 7.9609018999993468e+00
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  7.9609018999993468e+00
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 204 rows, 471 columns and 49769 nonzeros
Model fingerprint: 0xc9908bca
Model has 253 quadratic objective terms
Model has 484 simple general constraints
  484 INDICATOR
Variable types: 449 continuous, 22 integer (22 binary)
Coefficient statistics:
  Matrix range     [5e-09, 6e+01]
  Objective range  [1e+00, 1e+0

In [18]:
# 读取 UCI 数据
wholesale_customers = fetch_ucirepo(id=292) 

X_df = wholesale_customers.data.features
y_series = wholesale_customers.data.targets['Region']

# 清洗与对齐
X_df = X_df.replace('?', np.nan).dropna()
y_series = y_series.loc[X_df.index]

feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# 正确的标签编码
y_int = pd.to_numeric(y_series, errors='raise').values
y = np.where(y_int == 3, 1.0, -1.0)  # 3=1, 1，2=-1

# 假设你已准备好 X, y（y ∈ {-1,+1}）, 以及 feature_names（可选）
B, C = 5, 1.0
T_total = 120.0   # 两侧统一总时间（秒）
rho = 4

df_cmp = run_comparison(X, y, B, C, total_time=T_total, rho=rho, feature_names=feature_names, verbose=True)


Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 2.3527504350000527e+01
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  2.3527504350000527e+01
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 442 rows, 476 columns and 13214 nonzeros
Model fingerprint: 0xae32cabe
Model has 28 quadratic objective terms
Model has 49 simple general constraints
  49 INDICATOR
Variable types: 469 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e-06, 3e+02]
  Objective range  [1e+00, 1e+00]
  

In [19]:
# 读取 UCI 数据
breast_cancer_wisconsin_original = fetch_ucirepo(id=15)

X_df = breast_cancer_wisconsin_original.data.features
y_series = breast_cancer_wisconsin_original.data.targets['Class']

# 清洗与对齐
X_df = X_df.replace('?', np.nan).dropna()
y_series = y_series.loc[X_df.index]

feature_names = list(X_df.columns)
X = StandardScaler().fit_transform(X_df.values)

# 正确的标签编码（2/4 是整数）
y_int = pd.to_numeric(y_series, errors='raise').values
uniq = np.unique(y_int)
assert set(uniq) == {2, 4}, f"意外的类别：{uniq}"
y = np.where(y_int == 2, 1.0, -1.0)  # 2=+1, 4=-1

# 假设你已准备好 X, y（y ∈ {-1,+1}）, 以及 feature_names（可选）
B, C = 5, 1.0
T_total = 120.0   # 两侧统一总时间（秒）
rho = 4

df_cmp = run_comparison(X, y, B, C, total_time=T_total, rho=rho, feature_names=feature_names, verbose=True)


Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 1.5570850199997949e+01
Set parameter MIPGap to value 0.03
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  1.5570850199997949e+01
MIPGap  0.03
Heuristics  0.5
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 687 rows, 738 columns and 32119 nonzeros
Model fingerprint: 0x1328cec4
Model has 45 quadratic objective terms
Model has 81 simple general constraints
  81 INDICATOR
Variable types: 729 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [2e-03, 2e+01]
  Objective range  [1e+00, 1e+00]
  